In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import random as r
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, \
                             ExtraTreesClassifier, \
                             GradientBoostingClassifier, \
                             RandomForestClassifier 
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, \
                            precision_recall_fscore_support, \
                            roc_auc_score, \
                            classification_report

In [2]:
def breaker() -> None: print("\n" + 50*"*" + "\n")

In [3]:
class CFG(object):
    def __init__(self,
                 seed: int = 42,
                 n_splits: int = 5,
                 show_info: bool = False,
                 ):

        self.seed = seed
        self.n_splits = n_splits
        self.show_info = show_info
        self.features_path = "../input/4actrsfsna384"
        self.model_save_path = "models"
        if not os.path.exists(self.model_save_path): os.makedirs(self.model_save_path)

cfg = CFG(seed=42, show_info=True)

In [4]:
class Pipelines(object):
    def __init__(self, model_name: str, seed: int):
        self.model_name = model_name

        if self.model_name == "lgr":
            self.model = Pipeline(
                steps=[
                    ("classifier", LogisticRegression(random_state=seed, max_iter=1000)),
                ]
            )
        
        elif self.model_name == "knc":
            self.model = Pipeline(
                steps=[
                    ("classifier", KNeighborsClassifier()),
                ]
            )

        
        elif self.model_name == "dtc":
            self.model = Pipeline(
                steps=[
                    ("classifier", DecisionTreeClassifier(random_state=seed)),
                ]
            )

        elif self.model_name == "etc":
            self.model = Pipeline(
                steps=[
                    ("classifier", ExtraTreeClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "rfc":
            self.model = Pipeline(
                steps=[
                    ("classifier", RandomForestClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gbc":
            self.model = Pipeline(
                steps=[
                    ("classifier", GradientBoostingClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "abc":
            self.model = Pipeline(
                steps=[
                    ("classifier", AdaBoostClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "etcs":
            self.model = Pipeline(
                steps=[
                    ("classifier", ExtraTreesClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gnb":
            self.model = Pipeline(
                steps=[
                    ("classifier", GaussianNB()),
                ]
            )
        
        elif self.model_name == "xgb":
            self.model = Pipeline(
                steps=[
                    ("classifier", XGBClassifier(random_state=seed)),
                ]
            )

In [5]:
names: list = [
    "lgr", 
    "knc", 
    "rfc",
    "xgb"
]

model_names: list = [
    "swin_base_patch4_window12_384_in22k",
    "swin_large_patch4_window12_384_in22k",
    "vit_base_patch16_384",
    "vit_base_patch32_384",
    "vit_large_patch16_384",
    "vit_large_patch32_384",
    "deit3_base_patch16_384_in21ft1k",
    "deit3_large_patch16_384_in21ft1k",
    "deit3_small_patch16_384_in21ft1k",
    "deit_base_distilled_patch16_384",
]

df = pd.read_csv("../input/4ac-predlabel-dataframe/data.csv")
y = df.classes.copy().values

if cfg.show_info:
    breaker()
    for val in set(df.classes):
        print(f"Class {val} count : {df[df.classes == val].shape[0]}")

overall_best_acc: float = 0.0

for model_name in model_names:
    X = np.load(os.path.join(cfg.features_path, f"{model_name}.npy"))
    if cfg.show_info: 
        breaker()
        print(f"{model_name.upper()}")
    for name in names:
        fold: int = 1
        best_name: str = None
        best_acc: float = 0.0
        if cfg.show_info: breaker()
        for tr_idx, va_idx in StratifiedKFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True).split(X, y):
            X_train, X_valid, y_train, y_valid = X[tr_idx], X[va_idx], y[tr_idx], y[va_idx]
            my_pipeline = Pipelines(name, cfg.seed)
            my_pipeline.model.fit(X_train, y_train)

            y_pred = my_pipeline.model.predict(X_valid)
            if cfg.show_info:
                acc = accuracy_score(y_pred, y_valid)
                print(f"{my_pipeline.model_name}, {fold}, Accuracy : {acc:.5f}")
            else:
                acc = accuracy_score(y_pred, y_valid)

            if acc > overall_best_acc:
                overall_best_acc = acc
                overall_best_name = f"{model_name}_{name}_{fold}"
            
            if acc > best_acc:
                best_acc = acc
                best_name = f"{model_name}_{name}_{fold}"

            with open(os.path.join(cfg.model_save_path, f"{model_name}_{name}_{fold}.pkl"), "wb") as fp: pickle.dump(my_pipeline.model, fp)
            fold += 1
        
        print(f"\nBest : {best_name}")
    

if cfg.show_info: 
    breaker()
    print(f"Overall Best Model : {overall_best_name}")

breaker()


**************************************************

Class 0 count : 902
Class 1 count : 733
Class 2 count : 1130
Class 3 count : 764

**************************************************

SWIN_BASE_PATCH4_WINDOW12_384_IN22K

**************************************************

lgr, 1, Accuracy : 0.99575
lgr, 2, Accuracy : 0.99717
lgr, 3, Accuracy : 0.99717
lgr, 4, Accuracy : 0.99717
lgr, 5, Accuracy : 0.99433

Best : swin_base_patch4_window12_384_in22k_lgr_2

**************************************************

knc, 1, Accuracy : 0.99433
knc, 2, Accuracy : 0.99433
knc, 3, Accuracy : 0.99292
knc, 4, Accuracy : 0.99858
knc, 5, Accuracy : 0.98865

Best : swin_base_patch4_window12_384_in22k_knc_4

**************************************************

rfc, 1, Accuracy : 0.99292
rfc, 2, Accuracy : 0.99008
rfc, 3, Accuracy : 0.99008
rfc, 4, Accuracy : 0.99575
rfc, 5, Accuracy : 0.98298

Best : swin_base_patch4_window12_384_in22k_rfc_4

**************************************************

xgb, 1, Ac

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


lgr, 1, Accuracy : 0.57082


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


lgr, 2, Accuracy : 0.53966


...

lgr, 3, Accuracy : 0.54391


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


lgr, 4, Accuracy : 0.51275


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


lgr, 5, Accuracy : 0.56738

Best : deit_base_distilled_patch16_384_lgr_5

**************************************************

knc, 1, Accuracy : 0.43909
knc, 2, Accuracy : 0.41076
knc, 3, Accuracy : 0.40227
knc, 4, Accuracy : 0.39943
knc, 5, Accuracy : 0.39149

Best : deit_base_distilled_patch16_384_knc_1

**************************************************

rfc, 1, Accuracy : 0.47875
rfc, 2, Accuracy : 0.45751
rfc, 3, Accuracy : 0.45467
rfc, 4, Accuracy : 0.46601
rfc, 5, Accuracy : 0.46099

Best : deit_base_distilled_patch16_384_rfc_1

**************************************************

xgb, 1, Accuracy : 0.53541
xgb, 2, Accuracy : 0.47309
xgb, 3, Accuracy : 0.48442
xgb, 4, Accuracy : 0.52691
xgb, 5, Accuracy : 0.53617

Best : deit_base_distilled_patch16_384_xgb_5

**************************************************

Overall Best Model : swin_large_patch4_window12_384_in22k_lgr_2

**************************************************

